# Leave-One-Out Analysis
## Purpose:

* To assess the contribution and impact of each feature or data point.
* To identify features that significantly influence model performance or fairness.

## Process:

1. Step 1: Train the model on the entire dataset.
2. Step 2: For each feature or data point:
- Exclude that feature or data point from the dataset.
- Retrain the model on the reduced dataset.
- Measure the change in performance or fairness metrics.
3. Step 3: Compare the metrics with and without the excluded feature or data point to evaluate its impact.

## Applications:

Feature Importance: Determine which features are most influential in the model's predictions.
Fairness Analysis: Assess the impact of excluding each feature on fairness metrics to identify features that contribute to bias.
Model Validation: Validate the robustness and stability of the model by examining how sensitive it is to the removal of individual features or data points.

## Interpretation of Results
- Feature Importance: The calculated importance for each feature indicates how much the model's accuracy drops when that feature is excluded. Higher values suggest greater importance.
- Fairness Analysis: In the context of fairness, similar logic can be applied to see how fairness metrics change when each feature is excluded. This helps identify features that contribute to bias.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Dummy data
np.random.seed(0)
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, size=100)

# Train initial model
model = LogisticRegression()
model.fit(X, y)
initial_accuracy = accuracy_score(y, model.predict(X))
print(f'Initial Accuracy: {initial_accuracy}')

Initial Accuracy: 0.61


In [20]:
# Function to perform leave-one-out analysis
def leave_one_out_analysis(model, X, y):
    feature_importances = []
    initial_accuracy = accuracy_score(y, model.predict(X))

    for i in range(X.shape[1]):
        # Exclude one feature
        X_reduced = np.delete(X, i, axis=1)
        
        # Retrain model on reduced dataset
        model.fit(X_reduced, y)
        reduced_accuracy = accuracy_score(y, model.predict(X_reduced))
        
        # Calculate importance
        importance = initial_accuracy - reduced_accuracy
        feature_importances.append(importance)
        print(f'Feature {i} importance: {importance}')

    return feature_importances

# Perform leave-one-out analysis
leave_one_out_analysis(model, X, y)


Feature 0 importance: 0.020000000000000018
Feature 1 importance: -0.010000000000000009
Feature 2 importance: 0.010000000000000009
Feature 3 importance: 0.04999999999999993
Feature 4 importance: 0.010000000000000009


[0.020000000000000018,
 -0.010000000000000009,
 0.010000000000000009,
 0.04999999999999993,
 0.010000000000000009]

# Evaluation - fairness decision: 
## Key Variables
- loo_fair_value: Fairness value obtained by excluding one feature.
- baseline_fair_values_j: Baseline fairness value for the j-th feature.
- n: A reference value, 0 for cases where the metric is "difference" and 1 for "ratio"
- deltas_perf: Change in performance metric when excluding a feature.
- PerformanceMetrics.map_perf_metric_to_group: A mapping of performance metrics to their types (e.g., regression or classification).
- use_case_object.perf_metric_name: The name of the performance metric used in the use case.
- suggestion: The final suggestion based on the comparison (whether to include, exclude, or examine further).
- delta_conclusion: A string that accumulates conclusions based on the comparisons.

## Decision Logic
Comparing LOO Fairness Values with Baseline:

- If leaving out feature improves fairness, exlude it. If the absolute difference between the LOO fairness value and n is less than the absolute difference between the baseline fairness value and n, it indicates that excluding the feature improves fairness.
- Conversely, if the absolute difference between the LOO fairness value and n is greater, it indicates that excluding the feature worsens fairness - keep it.
- If the differences are equal, further examination is needed based on performance.
### Handling Regression and Classification Differently:

* For regression:
    * If excluding the feature does not worsen performance (deltas_perf <= 0), suggest 'exclude'.
    * Otherwise, suggest 'examine further'.
* For classification:
    * If excluding the feature improves performance (deltas_perf >= 0), suggest 'exclude'.
    * Otherwise, suggest 'examine further'.
* Annotations for Conclusions:
    * Depending on whether excluding the feature improves or worsens fairness, append " (+)" or " (-)" to delta_conclusion.

# Evaluation - tradeoff print out
## Initialization:

- i: A counter initialized to 1, used for numbering the output sections.
- p_var: A list of protected variables from the model parameters.
### Iterate Over Protected Variables:

- For each protected variable, format and print the results of the tradeoff analysis.
- Formatting Titles and Separators:

    - Create a title string for each protected variable.
    - Calculate a separator line to center the title within a line of 72 characters.
    - Print the formatted title and separator line.
### Print Trade-Off Analysis - Print sections for different threshold strategies:
- Single Threshold: Applies the same threshold to both privileged and unprivileged groups.
- Separated Thresholds: Applies different thresholds for privileged and unprivileged groups.
- Separated Thresholds under Neutral Fairness: Applies different thresholds with a tolerance for neutral fairness.
- Details of Each Section:

    - For each section, print the threshold values and the best performance metric value. Use the decimal_pts attribute to format the numeric output.
    - Print Additional Information:

    - If a replacement fairness metric is used, print a note indicating the replacement.


# `tradeoff.py`

## Purpose
- The TradeoffRate class computes the trade-off between performance and fairness metrics for a machine learning model. This analysis is crucial for understanding how different threshold settings impact the balance between model accuracy and fairness.

## Initialization
- The __init__ method initializes the class with several attributes, many of which are derived from the usecase_obj (an instance of a use case class).

## Key Attributes:

- perf_metric_name: Name of the performance metric used.
- fair_metric_name: Name of the fairness metric used.
- metric_group: Type of the fairness metric.
- p_var: List of protected variables.
- result: Dictionary to store computation results.
- fair_neutral_tolerance: Tolerance level for fairness-neutral trade-off. (It defines a range or threshold within which fairness metrics are considered to be "neutral" or acceptable. This tolerance is used to filter or constrain the results of trade-off analyses between performance and fairness metrics.)
- feature_mask: Masks for protected variables.
- map_metric_to_method: Mapping of metric names to their computation functions.
- sigma: Standard deviation for Gaussian smoothing.

## Methods
- compute_tradeoff(self, n_threads, tdff_pbar):
- This method computes the trade-off values and stores the results in the result attribute.

## Steps:

1. Determine mesh grids for thresholds.
2. For each protected variable, compute fairness and performance metrics.
3. Apply Gaussian smoothing to the fairness metrics.
4. Compute and store the best thresholds and associated performance metrics.
- _compute_max_perf(self, perf_grid, fair_grid):
5. Computes the best performance points based on different threshold settings:

- Single performance metric.
- Split performance metric.
- Fairness-constrained performance metric.

### Metric Computation Methods:
- These methods compute specific performance and fairness metrics on a grid. Examples include:

    - Balanced Accuracy: _compute_bal_accuracy_grid
    - F1 Score: _compute_f1_grid
    - Equal Opportunity: _compute_equal_opportunity_tr
    - Disparate Impact: _compute_disparate_impact_tr
    - Other Fairness Metrics: Various methods like _compute_demographic_parity_tr, _compute_false_discovery_rate_parity_tr, etc.
### Helper Methods:
- _compute_emp_lift_tr: Computes the empirical lift.
- _compute_expected_profit_tr: Computes the expected profit.
- _compute_rejected_harm_tr: Computes the harm from rejection.




## *gausian_smoothing*
- A bell-shaped curve that assigns higher weights to values near the center and lower weights to values further away. 
- This process effectively averages the data points, weighted by their distance from the center, resulting in a smoother dataset.
### Compute Fairness Values:

fair_values are computed based on the fairness metric selected.
- Apply Gaussian Smoothing:

    - gaussian_filter(fair_values, self.sigma) applies Gaussian smoothing to fair_values.
    - self.sigma is the standard deviation for the Gaussian kernel, controlling the extent of smoothing.
    - The result, fair_values_smooted, is a smoother version of fair_values, with reduced noise and finer details.
### Store Smoothed Fairness Values:
The smoothed fairness values are stored in the result dictionary for further analysis and decision-making.

### Why Gaussian Smoothing?
- Noise Reduction: Smoothing helps in reducing noise in the fairness metric values, which can be due to small fluctuations in the data or model predictions.
- Better Visualization: A smoother representation of fairness metrics makes it easier to visualize and interpret the trade-offs between performance and fairness.
- Improved Decision-Making: Smoothing can help in identifying more stable and reliable threshold settings by minimizing the impact of minor variations.
    - Imagine fairness metric values are plotted over different threshold settings. Without smoothing, the plot might look jagged and noisy, making it difficult to identify the optimal thresholds. Applying Gaussian smoothing would result in a smoother curve, highlighting the overall trend and making it easier to pinpoint the best threshold settings.

The method `_compute_fairness_metric_threshold` in the `CreditScoring` class is designed to compute a fairness metric threshold based on the `fair_threshold` attribute. This threshold is used to evaluate whether the model's predictions meet the specified fairness criteria.

### Detailed Explanation

#### Method Definition

```python
def _compute_fairness_metric_threshold(self, priv_m_v):
    """
    Computes the fairness metric threshold based on the fair_threshold variable

    Parameters
    ----------
    priv_m_v : float
            Privileged metric value

    Returns
    ----------
    fair_threshold : float
            Fairness metric threshold
    """
```

- **Purpose**: Computes the fairness metric threshold using the `fair_threshold` attribute.
- **Parameter**:
  - `priv_m_v`: The privileged metric value, which is a float.
- **Returns**:
  - `fair_threshold`: The computed fairness metric threshold, also a float.

#### Handling Different Variations of Threshold Value

```python
if self.fair_threshold > 1:
    self.fair_threshold = floor(self.fair_threshold)
```

- **Condition**: Checks if `fair_threshold` is greater than 1. This handles cases where `fair_threshold` might be provided as an integer percentage (e.g., 80 for 80%).
- **Action**: Uses `floor` to ensure `fair_threshold` is an integer. This effectively rounds down any decimal value to the nearest whole number.

#### Calculating the Fairness Metric Threshold

```python
if FairnessMetrics.map_fair_metric_to_group.get(self.fair_metric_name)[2] == 'ratio':
    fair_threshold = 1 - (self.fair_threshold / 100)
elif FairnessMetrics.map_fair_metric_to_group.get(self.fair_metric_name)[2] == 'difference':
    fair_threshold = (1 - (self.fair_threshold / 100)) * priv_m_v

return fair_threshold
```

- **Retrieving Metric Type**: The method looks up the type of the fairness metric (ratio or difference) from the `FairnessMetrics.map_fair_metric_to_group` dictionary.
- **Calculating Threshold**:
  - **For Ratio Metrics**: 
    - Computes the threshold as \(1 - \frac{\text{fair\_threshold}}{100}\).
    - Example: If `fair_threshold` is 80, the computed threshold will be \(1 - \frac{80}{100} = 0.2\).
  - **For Difference Metrics**:
    - Computes the threshold as \((1 - \frac{\text{fair\_threshold}}{100}) \times \text{priv\_m\_v}\).
    - Example: If `fair_threshold` is 80 and `priv_m_v` is 0.5, the computed threshold will be \((1 - \frac{80}{100}) \times 0.5 = 0.1\).

#### Handling Small Threshold Values

```python
else:
    return self.fair_threshold
```

- **Condition**: If `fair_threshold` is 1 or less, it is assumed to be already in a proper format (i.e., a float between 0 and 1).
- **Action**: Returns `fair_threshold` as is.

### Summary

The `_compute_fairness_metric_threshold` method calculates the threshold value for evaluating fairness based on the `fair_threshold` attribute and the type of fairness metric (ratio or difference). For ratio metrics, the threshold is computed as \(1 - \frac{\text{fair\_threshold}}{100}\). For difference metrics, it is computed as \((1 - \frac{\text{fair\_threshold}}{100}) \times \text{priv\_m\_v}\). If the `fair_threshold` is already a float between 0 and 1, it is returned directly. This method ensures that the fairness evaluation is consistent with the specified threshold criteria.